In [76]:
import math
import numpy as np

In [95]:
EPS = 1e-7
ITERATIONS = 200000

In [78]:
def _format_matrix(min, max, size):
    return np.array((max - min) * np.random.random((size, size)) + min, dtype=np.float64)
    
def rand_matrix(size):
    return np.array(np.random.random((size, size)), dtype=np.float64)

def diag_matrix(size):
    A = _format_matrix(0.0, 10.0, size)
    for i in range(size):
        A[i, i] = sum(A[i]) + 1
    return A

def hilbert_matrix(size):
    ans = np.array(np.zeros((size, size)), dtype=np.float64)
    for i in range(size):
        for j in range(size):
            ans[i, j] = 1.0 / (i + j + 1)
    return ans

def gen_right_column(size):
    return np.array(np.random.random(size), dtype=np.float64)

In [79]:
def equalVector(a, b, eps):
    return math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))])) < eps

In [126]:
def zeidelMethod(A, b):
    n = len(A)
    x = [0 for i in range(n)]
    xn = [100000 for i in range(n)]
    
    curIterations = 0
    while not equalVector(x, xn, EPS) and curIterations < ITERATIONS:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
    
    return xn

In [155]:
def zeidelKokorinTest2020CheckFunctionGavno(A, b):
    iterations = 0
    B = []
    d = []
    xn = []
    x = []
    
    for i in range (0, len(A)):
        d.append(b[i] / A[i][i])
        xn.append(d[i])
        x.append(d[i] + 1000)
        B.append([])
        for j in range (0, len(A[i])):
            B[i].append(-A[i][j] / A[i][i])
        B[i][i] = 0.0
    
    # detB = determinant(B)
    # eps = (1 - detB) * epsilon / detB
    eps = EPS
    
    while (not equalVector(x, xn, EPS)) and iterations < ITERATIONS:
        x = xn.copy()
        for i in range (0, len(B)):
            xn[i] = 0.0
            for j in range (0, len(B[i])):
                xn[i] += B[i][j] * (xn[j])
            xn[i] += d[i]
        iterations = iterations + 1
            
    
    return xn

In [137]:
def _test(method, matrix, column):
    return method(matrix, column)

def _check_eps(a, b, eps):
    return abs(a - b) < eps
EPSILON = 1e-2
def _check_ans(matrix, column, res, size):
    eps = EPSILON
    c = np.matmul(matrix, res)
    for i in range(size):
        if not _check_eps(c[i], column[i], eps):
            return False
    return True

def _testAll(matrix_gen, size, methods):
    matrix, column = matrix_gen(size), gen_right_column(size)
    print(f'Тест для матрицы {matrix} и столбца {column}')
    for method in methods:
        print(f'Результат выполнения метода {method.__name__}:')
        res = _test(method, matrix, column)
        check = _check_ans(matrix, column, res, size)
        if check:
            print(f'{res}. Погрешность не больше {EPSILON}\n')
        else:
            print(f'разошлось: matrix * res = {np.matmul(matrix, res)}\n ожидаемый столбец = {column}\n')

def _get_methods():
#     return [zeidelMethod]
    return [zeidelMethod]

def testDiagMatrix(size):
    _testAll(diag_matrix, size, _get_methods())

def testHilbertMatrix(size):
    _testAll(hilbert_matrix, size, _get_methods())
    
def testRandMatrix(size):
    _testAll(rand_matrix, size, _get_methods())

In [138]:
testDiagMatrix(10)

Тест для матрицы [[71.41718103  8.59128324  9.95692832  2.04917463  6.37724689  8.32026971
   8.33155005  7.27852556  1.71338605  8.66861602]
 [ 0.83262648 38.12661865  3.33673072  0.81420212  7.44792787  5.40098742
   7.78881026  0.8277912   2.82052054  1.77235023]
 [ 4.30167239  3.46565616 48.24435352  9.47624114  1.88853132  5.93445324
   8.31854337  3.27173655  0.2758333   5.96378975]
 [ 4.87958621  4.21739637  1.12953693 43.3803324   7.43043194  1.95671913
   5.65907603  3.0521435   7.71838916  1.95582393]
 [ 1.0634845   4.0269046   1.39846909  9.09565036 45.50448414  5.07616854
   5.30295674  5.72107344  9.33760205  0.91287676]
 [ 1.20013511  1.77723173  9.35988469  6.86706459  7.13683471 44.20700634
   3.4760208   2.46603527  3.08834972  4.22436778]
 [ 2.6190947   0.4137906   4.15599722  1.03983742  0.30124771  0.89617718
  39.81110584  8.90126413  6.18423152  9.52807673]
 [ 4.80574387  9.45525265  4.65705297  9.10566697  4.008688    0.97902886
   3.39128321 53.24289354  7.79234

In [153]:
testRandMatrix(5)

Тест для матрицы [[0.50446668 0.19609664 0.33127545 0.5446996  0.84737595]
 [0.75952257 0.33059246 0.68385996 0.32211936 0.23642307]
 [0.95127773 0.64752087 0.7978051  0.16064549 0.28351937]
 [0.26130678 0.2281669  0.43451397 0.53296546 0.79266664]
 [0.62104982 0.21506074 0.03111344 0.3614849  0.2450502 ]] и столбца [0.21686325 0.48939803 0.90480311 0.38277427 0.38028362]
Результат выполнения метода zeidelMethod:


/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


разошлось: matrix * res = [nan nan nan nan nan]
 ожидаемый столбец = [0.21686325 0.48939803 0.90480311 0.38277427 0.38028362]



In [154]:
testHilbertMatrix(5)

Тест для матрицы [[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]] и столбца [0.67163184 0.98302519 0.09482516 0.76702098 0.37585547]
Результат выполнения метода zeidelMethod:
[-1015.3715259708408, 18602.556245383825, -79220.25727471747, 118606.03018019565, -57649.95037872565]. Погрешность не больше 0.01

